Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [27]:
from random import choices
from dataclasses import dataclass
from copy import copy
from random import randint, random, choice
import lab9_lib

In [28]:
fitness = lab9_lib.make_problem(10)

for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")


print(fitness.calls)

10010100111111010001110100010010001101101001101000: 9.14%
10110110111100010001001010011001111101101111101011: 15.33%
11111100011000111001000011101101001101100110111001: 19.13%
11001100100010000000000000011000011010111001011010: 7.56%
10011101111111100110000110011001101101000011111111: 19.11%
10010100010011001100010110001010001101100011001010: 7.36%
00101000110110000011000111111010001100101100101000: 15.56%
01010101010100111001101101000001000101010001110011: 9.13%
01010011100001011010111101010000110001001010111100: 15.33%
11100100001011101001010000010100111111011100011010: 35.56%
10


In [29]:
POPULATION_SIZE = 30
OFFSPRING_SIZE = 20
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .15

SEQUENCE_LENGTH = 1000

In [30]:
@dataclass
class Individual:
    fitness: float
    genotype: list[bool]

### Selection

In [31]:
class Tournament():
    def __init__(self, size):
        self.size = size

    def select(self, pop):
        pool = [choice(pop) for _ in range(self.size)]
        champion = max(pool, key=lambda i: i.fitness)
        return champion


### Mutation

In [32]:
class BitFlip():
    def __init__(self, sequence_length):
        self.sequence_length = sequence_length
    
    def mutate(self, ind: Individual) -> Individual:
        offspring = copy(ind)
        pos = randint(0, self.sequence_length-1)
        offspring.genotype[pos] = not offspring.genotype[pos]
        offspring.fitness = None
        
        return offspring

### Recombination

In [33]:
class OnePointCrossover():
    def __init__(self, sequence_length):
        self.sequence_length = sequence_length

    def recombine(self,ind1: Individual, ind2: Individual) -> Individual:
        cut_point = randint(0, self.sequence_length-1)
        offspring = Individual(fitness=None,
                            genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
        assert len(offspring.genotype) == self.sequence_length
        return offspring

In [34]:
class EA_Simulation():
    def __init__(self, population_size, offspring_size, fitness_eval,selection, mutation, recombination, num_gen = 100, mutation_prob = 0.15):
        self.population_size = population_size
        self.offspring_size = offspring_size
        self.fitness_eval = fitness_eval
        self.select_parent = lambda pop : selection.select(pop)
        self.mutate = lambda ind : mutation.mutate(ind)
        self.recombine = lambda ind1, ind2: recombination.recombine(ind1, ind2)
        self.num_gen = num_gen
        self.mutation_prob = mutation_prob

    def set_inital_pop(self, population):
        self.population = population

    def simulate(self):
        record = []

        for _ in range(self.num_gen):
            offspring = list()
            for _ in range(OFFSPRING_SIZE):
                if random() < MUTATION_PROBABILITY:  # self-adapt mutation probability
                    # mutation  # add more clever mutations
                    p = self.select_parent(self.population)
                    o = self.mutate(p)
                else:
                    # xover # add more xovers
                    p1 = self.select_parent(self.population)
                    p2 = self.select_parent(self.population)
                    o = self.recombine(p1, p2)
                offspring.append(o)

            for i in offspring:
                i.fitness = fitness(i.genotype)
            self.population.extend(offspring)
            self.population.sort(key=lambda i: i.fitness, reverse=True)
            self.population = self.population[:self.population_size]

            record.append(self.population[0].fitness)

        return record
        

        
        

### Initialize Population

In [35]:
population = [
    Individual(
        genotype=[choice((False, False)) for _ in range(SEQUENCE_LENGTH)],
        fitness=None,
    )
    for _ in range(POPULATION_SIZE)
]

for i in population:
    i.fitness = fitness(i.genotype)

### Experiment

In [36]:
selection = Tournament(size = SEQUENCE_LENGTH)
mutation = BitFlip(sequence_length= SEQUENCE_LENGTH)
recombination = OnePointCrossover(sequence_length= SEQUENCE_LENGTH)

sim = EA_Simulation(population_size=POPULATION_SIZE, offspring_size= OFFSPRING_SIZE, fitness_eval=fitness, selection=selection, mutation=mutation, recombination=recombination)

sim.set_inital_pop(population)

record = sim.simulate()

In [37]:
record

[0.001,
 0.002,
 0.002,
 0.002,
 0.002,
 0.0029,
 0.0029,
 0.0038889999999999997,
 0.00789,
 0.009890000000000001,
 0.009890000000000001,
 0.0119,
 0.0119,
 0.0119,
 0.0119,
 0.0119,
 0.0119,
 0.0119,
 0.014,
 0.014,
 0.014,
 0.014,
 0.014,
 0.014,
 0.014,
 0.014,
 0.014,
 0.014,
 0.014,
 0.014,
 0.014,
 0.014,
 0.014,
 0.014,
 0.014,
 0.014,
 0.014,
 0.014,
 0.014,
 0.014,
 0.014,
 0.014,
 0.014,
 0.014,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.016,
 0.017779,
 0.017779,
 0.017779,
 0.02079,
 0.02079,
 0.02079,
 0.023899999999999998,
 0.023899999999999998,
 0.023899999999999998,
 0.023899999999999998,
 0.023899999999999998]